In [1]:
#
# %matplotlib notebook
from pathlib import Path
import numpy as np
import matplotlib
from matplotlib import cm
matplotlib.use("agg")
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
def visualize_syn(save_path):
    save_dir = (save_path / '..').resolve()
    save_dir.mkdir(parents=True, exist_ok=True)
    
    dpi, width, height = 50, 2000, 1000
    figsize = width / float(dpi), height / float(dpi)
    LabelSize, font_gap = 30, 4
    
    fig = plt.figure(figsize=figsize)
    
    times = np.arange(0, np.pi * 100, 0.1)
    num = len(times)
    x = []
    for i in range(num):
        scale = (i + 1.) / num * 4
        value = times[i] * scale
        x.append(np.sin(value) * (1.3 - scale))
    x = np.array(x)
    y = np.cos( x * x - 0.3 * x )
    
    cur_ax = fig.add_subplot(2, 1, 1)
    cur_ax.plot(times, x)
    cur_ax.set_xlabel("time", fontsize=LabelSize)
    cur_ax.set_ylabel("x", fontsize=LabelSize)
    for tick in cur_ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        tick.label.set_rotation(30)
    for tick in cur_ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        
    
    cur_ax = fig.add_subplot(2, 1, 2)
    cur_ax.plot(times, y)
    cur_ax.set_xlabel("time", fontsize=LabelSize)
    cur_ax.set_ylabel("f(x)", fontsize=LabelSize)
    for tick in cur_ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        tick.label.set_rotation(30)
    for tick in cur_ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(LabelSize - font_gap)
        
    # fig.tight_layout()
    # plt.subplots_adjust(wspace=0.05)#, hspace=0.4)
    fig.savefig(save_path, dpi=dpi, bbox_inches="tight", format="pdf")
    plt.close("all")
    # plt.show()

In [3]:
# Visualization
home_dir = Path.home()
desktop_dir = home_dir / 'Desktop'
print('The Desktop is at: {:}'.format(desktop_dir))
visualize_syn(desktop_dir / 'tot-synthetic-v0.pdf')

The Desktop is at: /Users/xuanyidong/Desktop
